# Distributional Semantics


For this notebook, we'll be using the 500 document Brown corpus included in NLTK

In [ ]:
from nltk.corpus import brown

This notebook is divided up into two independent parts: the first uses PMI for distinguishing good collocations, and the second involves building a vector space model for document retrieval.

For the PMI portion, we'll use a function which extracts the information we need for a particular two word collocation, namely counts of each word individually, counts of the collocation, and the total number of word tokens in the corpus, and then calculates the PMI:

In [ ]:
import math

def get_PMI_for_collocation_brown(word1,word2):
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_count = 0.0 # so that division results in a float
    for sent in brown.sents():
        sent = [word.lower() for word in sent]
        for i in range(len(sent)):
            total_count += 1
            if sent[i] == word1:
                word1_count += 1
                if i < len(sent) - 1 and sent[i + 1] == word2:
                    both_count += 1
            elif sent[i] == word2:
                word2_count += 1
    return math.log((both_count/total_count)/((word1_count/total_count)*(word2_count/total_count)), 2)
                
        

Note that in a typical use case, we probably wouldn't do it this way, since we'd likely want to calculate PMI across many different words, and collecting the statisitcs for this can be done in a single pass across the corpus for all words, and then the PMI calculated in a separate function. Anyway, let's compare the PMI for two phrases, "hard work" and "some work"

In [ ]:
print get_PMI_for_collocation_brown("hard","work")
print get_PMI_for_collocation_brown("some","work")

Based on PMI, "hard work" appears to be a much better collocation than "some work", which matches our intuition. Go ahead and try out this out some other collocations. 

For the second part of the notebook, let's create a sparse document-term matrix, using sci-kit learn. We will do a document-term matrix rather than term-document because we will be performing SVD dimensionality reduction to produce dense document representations for document retrevial. Note that this is actually identical to creating a BOW feature representation for each document; the difference comes in how we used the representation. 

In [ ]:
from sklearn.feature_extraction import DictVectorizer

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word.lower()] = BOW.get(word.lower(),0) + 1
    return BOW

texts = []
for fileid in brown.fileids():
    texts.append(get_BOW(brown.words(fileid)))

vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(texts)
print brown_matrix


Our matrix is sparse: for instance, columns 0-48 in row 0 are empty, and are just left out, only the rows and columns with values other than zeros are displayed

Rather than removing stopwords as we did for text classification, let's add some idf weighting to this matrix. Scikit-learn has a built-in tf-idf transformer for just this purpose.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False,norm=None)

brown_matrix = transformer.fit_transform(brown_matrix)

print brown_matrix

Next, let's apply SVD. Scikit-learn does not expose the internal details of the decomposition, we just use the TruncatedSVD class directly get a matrix with k dimensions. Since the Brown corpus is a fairly small corpus, we'll do k=100

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)
brown_matrix = svd.fit_transform(brown_matrix)

print brown_matrix

Note that this matrix is not sparse.

The last thing we'll do is build a very simple document retrevial system based on the vector space model we've built: it will take some query input, apply all the transformations we have defined above, then find the Brown document with the highest cosine similarity to the query document. Here we are using scipy's cosine distance function; we actually find the smallest distance instead of the largest similarity.

In [ ]:
from scipy.spatial.distance import cosine as cos_distance

def transform_query(query_text):
    return svd.transform(transformer.transform(vectorizer.transform([get_BOW(query_text.split())])))[0]

def get_best_doc_num(query):
    f = lambda x: cos_distance(query,brown_matrix[x])
    best_doc = min(range(500),key=f)
    return best_doc

Let's test this out with a couple of sets of key words, with the idea of getting a religious text in the first example, and a mathematics text in the second (the Brown corpus has both). We'll also look at the specific vectors and distances involved.

In [ ]:
def try_query(query_text):
    query = transform_query(query_text)
    doc_num = get_best_doc_num(query)
    print "query text"
    print query_text
    print "query vector"
    print query
    print "best document vector"
    print brown_matrix[doc_num]
    print "cosine distance from query to document"
    print cos_distance(query,brown_matrix[doc_num])
    print "best document sample"
    print brown.words(brown.fileids()[doc_num])[:50]

try_query("heaven hell devil lord")
try_query("matrix algebra eigenvalue")